In [1]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random


  Activating project at `~/code/CT_MPS_mini/CT`


In [ ]:
L = 4
# Generate list of right translations of 1:L more efficiently
# Using array comprehension and avoiding modulo operations where possible
translations = [circshift(1:L, shift) for shift in 0:L-1]

function fold(translation, ancilla::Int)
    if ancilla ==0
        ram_phy = [i for pairs in zip(translation[1:(L÷2)], reverse(translation[(L÷2+1):L])) for i in pairs]
    elseif ancilla ==1
        ram_phy = vcat(L+1,[i for pairs in zip(translations[1:(L÷2)], reverse(translations[(L÷2+1):L])) for i in pairs])
    elseif ancilla ==2
        error("Not implemented yet")
    end

    # phy_ram[physical] = actual in ram
    phy_ram = fill(0, L+ancilla)
    for (ram, phy) in enumerate(ram_phy)
        phy_ram[phy] = ram
    end

    return ram_phy, phy_ram
end

conn_pairs = Dict()
for (i1, translation) in enumerate(translations)
    # println(fold(translation, 0))
    ram_phy, phy_ram = fold(translation, 0)
    # println(ram_phy)
    pairs = []
    for i in reverse(collect(1:L)[2:end])
        current_pos = findfirst(x -> x == i, ram_phy)
        next_pos = findfirst(x -> x == (i-1)%L, ram_phy)
        pair = (current_pos, next_pos)
        push!(pairs, pair)
        # println(i, " is in position $current_pos carry into $(i-1) which is in position $next_pos")
    end

    conn_pairs[i1] = pairs
end


4 is in position 2 carry into 3 which is in position 4
3 is in position 4 carry into 2 which is in position 3
2 is in position 3 carry into 1 which is in position 1
4 is in position 1 carry into 3 which is in position 2
3 is in position 2 carry into 2 which is in position 4
2 is in position 4 carry into 1 which is in position 3
4 is in position 3 carry into 3 which is in position 1
3 is in position 1 carry into 2 which is in position 2
2 is in position 2 carry into 1 which is in position 4
4 is in position 4 carry into 3 which is in position 3
3 is in position 3 carry into 2 which is in position 1
2 is in position 1 carry into 1 which is in position 2
